# Kaggle Competition Example

In [4]:
import os
import zipfile
from snowflake.snowpark import Session
import pandas as pd


import os
from kaggle.api.kaggle_api_extended import KaggleApi
from typing import List


In [ ]:

class InstacartDataLoader:
    def __init__(self, session: Session):
        self.session = session
        
    def unzip_data_files(self, path: str):
        """Unzip all CSV files in the directory"""
        print("Unzipping data files...")
        for file in os.listdir(path):
            if file.endswith('.zip'):
                zip_path = os.path.join(path, file)
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(path)
                print(f"Unzipped: {file}")
                
    def load_csv_to_snowflake(self, file_path: str, table_name: str, schema: str = 'INSTACART_RAW'):
        """Load CSV file to Snowflake table"""
        print(f"Loading {file_path} to {schema}.{table_name}")
        chunk_size = 100000
        for chunk in pd.read_csv(file_path, chunksize=chunk_size):
            df = self.session.create_dataframe(chunk)
            df.write.save_as_table(f"{schema}.{table_name}", mode="append")
            
    def setup_incremental_loading(self):
        """Setup tables for incremental loading"""
        self.session.sql("""
            CREATE OR REPLACE TABLE INSTACART_RAW.ORDER_PRODUCTS_STAGE (
                order_id INTEGER,
                product_id INTEGER,
                add_to_cart_order INTEGER,
                reordered INTEGER,
                file_name VARCHAR,
                loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
            )
        """).collect()


In [14]:


def download_instacart_data(path: str = "./data") -> List[str]:
    """
    Download Instacart Market Basket Analysis data
    
    Args:
        path: Directory to save the data
        
    Returns:
        List of downloaded file paths
    
    Raises:
        Exception: If competition rules haven't been accepted or other API errors
    """
    try:
        api = KaggleApi()
        api.authenticate()
        
        # Create data directory if it doesn't exist
        os.makedirs(path, exist_ok=True)
        
        # Check if we've already downloaded the data
        zip_path = f"{path}/instacart-market-basket-analysis.zip"
        if os.path.exists(zip_path):
            print("Data already downloaded, using existing files...")
        else:
            print("Downloading competition data...")
            try:
                api.competition_download_files(
                    'instacart-market-basket-analysis',
                    path=path
                )
            except Exception as e:
                if "rules" in str(e):
                    raise Exception(
                        "Please accept the competition rules first at "
                        "https://www.kaggle.com/competitions/instacart-market-basket-analysis"
                    ) from e
                raise
        
        # Unzip if needed
        csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]
        if not csv_files:
            print("Extracting files...")
            import zipfile
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(path)
        
        # List available files
        csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]
        print("\nAvailable files:")
        for file in csv_files:
            print(f"- {file}")
            
        return [os.path.join(path, f) for f in csv_files]
        
    except Exception as e:
        print(f"Error downloading data: {str(e)}")
        raise


In [15]:

# Usage
try:
    files = download_instacart_data()
    print(f"\nSuccessfully downloaded {len(files)} files")
except Exception as e:
    print(f"Failed to download data: {str(e)}")


Data already downloaded, using existing files...

Available files:
- sample_submission.csv

Successfully downloaded 1 files


In [18]:
! unzip -o ./data/instacart-market-basket-analysis.zip -d ./data

Archive:  ./data/instacart-market-basket-analysis.zip
  inflating: ./data/sample_submission.csv  


In [16]:

class InstacartDataLoader:
    def __init__(self, session: Session, data_path: str = "./data"):
        self.session = session
        self.data_path = data_path
        
    @staticmethod
    def prep_df(df):
        """Prepare DataFrame for Snowflake loading"""
        df = df.reset_index(drop=True)
        df.index = pd.RangeIndex(start=0, stop=len(df))
        df.columns = [col.upper() for col in df.columns]
        return df
        
    def load_data(self):
        """Load all data using Snowpark DataFrames"""
        print("Loading data...")
        
        # Define all tables and their primary keys
        tables_info = {
            "aisles.csv": ("AISLES", ["AISLE_ID"]),
            "departments.csv": ("DEPARTMENTS", ["DEPARTMENT_ID"]),
            "products.csv": ("PRODUCTS", ["PRODUCT_ID"]),
            "orders.csv": ("ORDERS", ["ORDER_ID"]),
            "order_products__prior.csv": ("ORDER_PRODUCTS", ["ORDER_ID", "PRODUCT_ID"]),
            "order_products__train.csv": ("ORDER_PRODUCTS", ["ORDER_ID", "PRODUCT_ID"])
        }
        
        # Create schema
        self.session.sql("CREATE SCHEMA IF NOT EXISTS INSTACART_RAW").collect()
        
        # Load each file
        for file, (table, pk_cols) in tables_info.items():
            print(f"Processing {file}...")
            
            # Read and prep data
            df = self.prep_df(pd.read_csv(f"{self.data_path}/{file}"))
            
            # Create or append to table
            mode = "overwrite" if not file.startswith("order_products") else "append"
            self.session.create_dataframe(df) \
                .write.mode(mode) \
                .save_as_table(f"INSTACART_RAW.{table}")
            
            # Add primary key if not appending
            if mode == "overwrite":
                pk_str = ", ".join(pk_cols)
                try:
                    self.session.sql(f"""
                        ALTER TABLE INSTACART_RAW.{table} 
                        ADD PRIMARY KEY ({pk_str})
                    """).collect()
                except Exception as e:
                    print(f"Warning: Could not add primary key to {table}: {str(e)}")
    
    def validate_load(self):
        """Validate the loaded data"""
        print("\nValidating loaded data...")
        for table in ["AISLES", "DEPARTMENTS", "PRODUCTS", "ORDERS", "ORDER_PRODUCTS"]:
            count = self.session.sql(f"SELECT COUNT(*) as cnt FROM INSTACART_RAW.{table}").collect()[0]['CNT']
            print(f"{table}: {count:,} rows")


In [12]:
from snowflake_feature_store.connection import get_connection

# Specify all connection parameters directly
database = "DATASCIENCE"
schema = "INSTACART_RAW"
warehouse = "DS_WH_XS"
role = "DATA_SCIENTIST"

# Get connection with all custom parameters
conn = get_connection(
    database=database, 
    schema=schema, 
    warehouse=warehouse,
    role=role
)

# Usage
loader = InstacartDataLoader(conn.session)
loader.unzip_data_files('./data/')


2025-02-26 19:17:16,403 - snowflake_feature_store - INFO - Using active Snowflake session
2025-02-26 19:17:16,407 - snowflake_feature_store - INFO - Initialized connection to "DATASCIENCE"."INSTACART_RAW"
2025-02-26 19:17:19,182 - snowflake_feature_store - INFO - Using role: "DATA_SCIENTIST", warehouse: "DS_WH_XS", database: DATASCIENCE, schema: INSTACART_RAW
Unzipping data files...
Unzipped: instacart-market-basket-analysis.zip


In [ ]:
loader.load_data()
loader.validate_load()
